# Import Data

In [1]:
import pandas as pd
import numpy as np
X_test= pd.read_csv('input_test.csv')
X_train_raw = pd.read_csv('input_train.csv')
y_random_raw = pd.read_csv('y_rand.csv')
y_train_raw = pd.read_csv('y_train.csv')

# Small preprocessing step

In [2]:
#REPLACING MISSING VALUES WITH THE MEDIAN VALUE

for i in X_train_raw.columns:
    median = np.nanmedian(np.array(X_train_raw[[i]]).flatten())
    X_train_raw[[i]] = X_train_raw[[i]].fillna(median)
    median = np.nanmedian(np.array(X_test[[i]]).flatten())
    X_test[[i]] = X_test[[i]].fillna(median)

# Assigning each target with its observation

In [3]:
X_train_raw.assign(target = np.nan)
for cluster in range (0,1464):
    for day in range (0,21):
        i = cluster
        j = day
        if not y_train_raw.loc[y_train_raw['sample_id'] == cluster*21 + day,'target'].empty:
            X_train_raw.loc[(X_train_raw['cluster']==cluster)&(X_train_raw['day'] == day), 'target'] = y_train_raw.loc[y_train_raw['sample_id'] == cluster*21 + day,'target'].values[0]
X_train = X_train_raw.loc[X_train_raw['target'].notna()]
X_train.fillna(0)

,id,cluster,day,asset,md,bc,ret_1,ret_2,ret_3,ret_4,...,ret_15,ret_16,ret_17,ret_18,ret_19,ret_20,ret_21,ret_22,ret_23,target
0,0,0,17,10,-5.734573,0.125855,-0.002060,0.005848,-0.002736,-0.005487,...,-0.000339,0.001018,-0.002372,-0.000340,-0.000680,-0.007140,0.006849,0.002041,-0.001697,0.006903
1,1,0,0,13,-5.813696,0.418711,-0.042280,-0.009460,-0.005173,-0.006000,...,-0.005726,0.006993,-0.003268,0.015574,-0.010492,0.004486,0.011368,0.001606,0.000401,0.000301
2,2,0,20,12,-5.814399,0.500111,0.003665,-0.010841,0.007557,0.004981,...,-0.000170,-0.002833,-0.007442,-0.001832,-0.005677,-0.007093,0.001626,0.004349,-0.004677,-0.001543
3,3,0,19,12,-5.814399,0.500111,-0.018364,0.003789,0.011795,0.016029,...,-0.009794,-0.001023,-0.002959,0.002568,-0.005579,0.001317,-0.010005,0.004677,-0.004196,0.011333
4,4,0,18,12,-5.814399,0.500111,-0.001589,0.011711,-0.010507,-0.002555,...,-0.018505,-0.010855,0.002022,0.001211,-0.002648,0.001559,-0.001153,0.000635,0.001961,-0.004069
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
491033,491033,1463,7,4,-8.219044,0.504312,-0.011203,0.002543,-0.001614,-0.006468,...,-0.001828,0.000462,0.009613,0.003174,0.003164,0.045506,0.004309,-0.031109,0.004429,-0.000977
491034,491034,1463,8,4,-8.219044,0.504312,-0.000611,0.013210,-0.013038,0.006605,...,-0.031028,-0.035453,-0.018734,0.008217,0.000959,-0.019636,0.004641,0.004376,-0.003147,0.003541
491035,491035,1463,9,4,-8.219044,0.504312,-0.002688,-0.006124,0.005423,-0.011277,...,0.002543,-0.005535,-0.004638,0.002097,-0.013485,-0.009192,0.007136,-0.008030,0.010714,0.002779
491036,491036,1463,17,3,-7.648535,0.622304,0.000211,-0.023664,0.000866,-0.009081,...,0.007603,0.006057,0.002429,0.005269,-0.007338,-0.000739,0.005918,-0.012922,-0.008727,0.001465


In [4]:
y_train = X_train.pop('target')

# Dropping useless features 

In [5]:
X_train = X_train.drop(['md','id', 'asset','cluster','day'], axis = 1)

In [6]:
X_test_pred = X_test.drop(['md','id', 'asset','cluster','day'], axis = 1)

# Example of randomized search with XGBoost

In [14]:
import xgboost as xgb
from sklearn import metrics
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split

In [ ]:
model = xgb.XGBRegressor()
hyperparameter_grid = {
    'n_estimators': [500],
    'max_depth': [20,10],
    'learning_rate': [0.05,0.045,0.055],
    'min_child_weight': [10,1],
    'subsample':[0.75, 0.5],
    'colsample_bytree':[0.75,0.5],
    'gamma': [0],
    'reg_lambda':[0,0.3],
    'alpha':[0,0.3]
    }
random_cv = RandomizedSearchCV(estimator=model,
            param_distributions=hyperparameter_grid, 
            cv=2, n_iter=200,
            scoring = 'neg_root_mean_squared_error',n_jobs = 1,
            return_train_score = True, verbose = 5,
            random_state=42)
random_cv.fit(X_train,y_train)
 

C:\Users\victo\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:306: UserWarning: The total space of parameters 192 is smaller than n_iter=200. Running 192 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Fitting 2 folds for each of 192 candidates, totalling 384 fits
[CV 1/2] END alpha=0, colsample_bytree=0.75, gamma=0, learning_rate=0.05, max_depth=20, min_child_weight=10, n_estimators=500, reg_lambda=0, subsample=0.75;, score=(train=-0.003, test=-0.007) total time= 8.3min
[CV 2/2] END alpha=0, colsample_bytree=0.75, gamma=0, learning_rate=0.05, max_depth=20, min_child_weight=10, n_estimators=500, reg_lambda=0, subsample=0.75;, score=(train=-0.003, test=-0.007) total time= 7.6min
[CV 1/2] END alpha=0, colsample_bytree=0.75, gamma=0, learning_rate=0.05, max_depth=20, min_child_weight=10, n_estimators=500, reg_lambda=0, subsample=0.5;, score=(train=-0.003, test=-0.007) total time= 6.5min
[CV 2/2] END alpha=0, colsample_bytree=0.75, gamma=0, learning_rate=0.05, max_depth=20, min_child_weight=10, n_estimators=500, reg_lambda=0, subsample=0.5;, score=(train=-0.003, test=-0.007) total time= 6.5min
[CV 1/2] END alpha=0, colsample_bytree=0.75, gamma=0, learning_rate=0.05, max_depth=20, min_chi

[CV 2/2] END alpha=0, colsample_bytree=0.75, gamma=0, learning_rate=0.045, max_depth=20, min_child_weight=10, n_estimators=500, reg_lambda=0.3, subsample=0.5;, score=(train=-0.004, test=-0.007) total time= 6.1min
[CV 1/2] END alpha=0, colsample_bytree=0.75, gamma=0, learning_rate=0.045, max_depth=20, min_child_weight=1, n_estimators=500, reg_lambda=0, subsample=0.75;, score=(train=-0.002, test=-0.007) total time= 7.4min
[CV 2/2] END alpha=0, colsample_bytree=0.75, gamma=0, learning_rate=0.045, max_depth=20, min_child_weight=1, n_estimators=500, reg_lambda=0, subsample=0.75;, score=(train=-0.002, test=-0.007) total time= 7.2min
[CV 1/2] END alpha=0, colsample_bytree=0.75, gamma=0, learning_rate=0.045, max_depth=20, min_child_weight=1, n_estimators=500, reg_lambda=0, subsample=0.5;, score=(train=-0.002, test=-0.007) total time= 6.8min
[CV 2/2] END alpha=0, colsample_bytree=0.75, gamma=0, learning_rate=0.045, max_depth=20, min_child_weight=1, n_estimators=500, reg_lambda=0, subsample=0.5;

[CV 1/2] END alpha=0, colsample_bytree=0.75, gamma=0, learning_rate=0.055, max_depth=20, min_child_weight=1, n_estimators=500, reg_lambda=0.3, subsample=0.5;, score=(train=-0.002, test=-0.007) total time= 6.7min
[CV 2/2] END alpha=0, colsample_bytree=0.75, gamma=0, learning_rate=0.055, max_depth=20, min_child_weight=1, n_estimators=500, reg_lambda=0.3, subsample=0.5;, score=(train=-0.003, test=-0.007) total time= 6.3min
[CV 1/2] END alpha=0, colsample_bytree=0.75, gamma=0, learning_rate=0.055, max_depth=10, min_child_weight=10, n_estimators=500, reg_lambda=0, subsample=0.75;, score=(train=-0.006, test=-0.008) total time= 3.4min
[CV 2/2] END alpha=0, colsample_bytree=0.75, gamma=0, learning_rate=0.055, max_depth=10, min_child_weight=10, n_estimators=500, reg_lambda=0, subsample=0.75;, score=(train=-0.006, test=-0.008) total time= 3.4min
[CV 1/2] END alpha=0, colsample_bytree=0.75, gamma=0, learning_rate=0.055, max_depth=10, min_child_weight=10, n_estimators=500, reg_lambda=0, subsample=

In [ ]:
pd.DataFrame(random_cv.best_estimator_.feature_importances_, index=X_train.columns).plot.bar(figsize=(30, 8))
#pd.DataFrame(model.feature_importances_, index=X_train.columns).plot.bar(figsize=(30, 8))

In [ ]:
print(random_cv.best_params_)
results = pd.DataFrame(random_cv.cv_results_)
results.sort_values('rank_test_score')

In [8]:
model.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.75,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
             grow_policy='depthwise', importance_type=None,
             interaction_constraints='', learning_rate=0.04, max_bin=256,
             max_cat_threshold=64, max_cat_to_onehot=4, max_delta_step=0,
             max_depth=90, max_leaves=0, min_child_weight=20, missing=nan,
             monotone_constraints='()', n_estimators=4000, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, ...)

# Stacked Model

In [8]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import StackingRegressor
from sklearn.ensemble import RandomForestRegressorb
import xgboost as xb
import lightgbm as lgb
xgb = xb.XGBRegressor(booster = 'gbtree',colsample_bytree=0.75, gamma=0, learning_rate=0.04, max_depth=90, min_child_weight=20, n_estimators=4000, subsample=1)
rf = RandomForestRegressor(bootstrap=True, max_depth=84, max_features='sqrt', min_samples_leaf=9, min_samples_split=6, n_estimators=200)
LGB = lgb.LGBMRegressor(boosting_type= 'goss', learning_rate= 0.29154431891537524, max_depth= 104, min_data=33, num_leaves= 150, objective= 'regression', reg_alpha= 0.10292247147901223, reg_lambda= 4.8495492608099715, sub_feature= 0.8324426408004217)
base_models = [
    ('XGB', xgb),
    ('RF', rf),
    ('LGB', LGB)
    ]
stacked = StackingRegressor(
    estimators = base_models,
    final_estimator = LinearRegression())
  
stacked.fit(X_train, y_train)

[LightGBM] [Warning] min_data_in_leaf is set with min_child_samples=20, will be overridden by min_data=33. Current value: min_data_in_leaf=33
[LightGBM] [Warning] feature_fraction is set with colsample_bytree=1.0, will be overridden by sub_feature=0.8324426408004217. Current value: feature_fraction=0.8324426408004217
[LightGBM] [Warning] min_data_in_leaf is set with min_child_samples=20, will be overridden by min_data=33. Current value: min_data_in_leaf=33
[LightGBM] [Warning] feature_fraction is set with colsample_bytree=1.0, will be overridden by sub_feature=0.8324426408004217. Current value: feature_fraction=0.8324426408004217
[LightGBM] [Warning] min_data_in_leaf is set with min_child_samples=20, will be overridden by min_data=33. Current value: min_data_in_leaf=33
[LightGBM] [Warning] feature_fraction is set with colsample_bytree=1.0, will be overridden by sub_feature=0.8324426408004217. Current value: feature_fraction=0.8324426408004217
[LightGBM] [Warning] min_data_in_leaf is se

StackingRegressor(estimators=[('XGB',
                               XGBRegressor(base_score=None, booster='gbtree',
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=0.75,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric=None,
                                            feature_types=None, gamma=0,
                                            gpu_id=None, grow_policy=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_r...
                               RandomForestRegressor(max_depth=84,
                                                     max_features='sqrt',
                                                     min_samples_leaf=9,
                                                     min_samples_split=6,
                                                     n_estimators=200)),
                              ('LGB',
                               LGBMRegressor(boosting_type='goss',
                                             learning_rate=0.29154431891537524,
                                             max_depth=104, min_data=33,
                                             num_leaves=150,
                                             objective='regression',
                                             reg_alpha=0.10292247147901223,
                                             reg_lambda=4.8495492608099715,
                                             sub_feature=0.8324426408004217))],
                  final_estimator=LinearRegression())

# Creating submission file

In [9]:
y_pred = stacked.predict(X_test_pred)

In [10]:
X_test[['target']] = y_pred.reshape(len(y_pred),1)
a = {'sample_id':[]}
a['target'] = []
for cluster in range(1464,2091):
    for day in range(0,21):
        if not y_random_raw.loc[y_random_raw['sample_id']==cluster*21 +day,'target'].empty:
            a['sample_id'].append(cluster*21+day)
            a['target'].append(X_test.loc[(X_test['cluster'] == cluster) & (X_test['day'] == day), 'target'].values[0])
        
submission = pd.DataFrame(a)

,sample_id,target
0,30744,0.004072
1,30745,-0.001882
2,30746,0.002144
3,30747,0.025043
4,30748,-0.007172
...,...,...
13128,43906,0.001243
13129,43907,-0.020267
13130,43908,-0.001842
13131,43909,-0.003183


In [12]:
submission.to_csv('Submission.csv',header = True, index = False)